In [1]:
# ============================================================================
# CELDA 1: CONFIGURACIÓN E IMPORTS
# ============================================================================
%run ./00_template.py

import os
import time
import requests
import geopandas as gpd
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime
from dateutil import parser as dtparser

# Configuración de Salida
OUTPUTS_DIR.mkdir(parents=True, exist_ok=True)
print(f"📂 Carpeta de salida verificada: {OUTPUTS_DIR}")

# Configuración de OTP
OTP_HOST = os.getenv("OTP_URL", "http://otp:8080").rstrip("/")
OTP_URL = f"{OTP_HOST}/otp/routers/default/plan"

# FECHA CLAVE: Usamos un día laboral normal (Martes 8:30 AM)
# Navidad (25-Dic) tiene frecuencias de feriado, no sirve para evaluar desiertos reales.
WHEN_ISO = "2023-10-18T08:30:00-03:00"

print(f"📡 Endpoint OTP: {OTP_URL}")
print(f"📅 Fecha simulada: {WHEN_ISO}")

=== 🌍 Template PEP1 Configurado Exitosamente ===
📂 Base Dir       : /home/jovyan
💾 GeoDatabase    : ✅ Encontrada
📊 Censo CSV      : ✅ Encontrado
📂 Carpeta de salida verificada: /home/jovyan/outputs
📡 Endpoint OTP: http://otp:8080/routers/default/plan/otp/routers/default/plan
📅 Fecha simulada: 2023-10-18T08:30:00-03:00


In [2]:
# ============================================================================
# CELDA 2: MOTOR DE CONSULTAS OTP (Hora fija insertada como texto)
# ============================================================================
import requests
import time
from dateutil import parser as dtparser

OTP_URL = "http://otp:8080/otp/gtfs/v1"

# 1. Quitamos $time de la cabecera
# 2. Usamos dos %s: Uno para la hora, otro para los modos
PLAN_QUERY = """
query($oLat: CoordinateValue!, $oLon: CoordinateValue!, $dLat: CoordinateValue!, $dLon: CoordinateValue!) {
  planConnection(
    origin: { location: { coordinate: { latitude: $oLat, longitude: $oLon } } }
    destination: { location: { coordinate: { latitude: $dLat, longitude: $dLon } } }
    dateTime: { earliestDeparture: "%s" }
    modes: {
      direct: [WALK]
      transit: { transit: [%s] }
    }
  ) {
    edges {
      node {
        start
        end
        legs {
          mode
          duration
        }
      }
    }
  }
}
"""

def otp_graphql(query: str, variables: dict, timeout=60, retries=3):
    """Envía la consulta a OTP."""
    for attempt in range(retries + 1):
        try:
            r = requests.post(OTP_URL, json={"query": query, "variables": variables}, timeout=timeout)
            r.raise_for_status()
            data = r.json()
            if "errors" in data:
                # Imprimir el mensaje de error para debug
                print(f"⚠️ OTP Error: {data['errors'][0]['message']}")
                return None
            return data["data"]
        except Exception as e:
            if attempt < retries:
                time.sleep(0.5)
                continue
            return None

def otp_travel_time_minutes(o_lat, o_lon, d_lat, d_lon, when_iso, modes=["BUS", "SUBWAY", "RAIL"]):
    """
    Calcula tiempo de viaje.
    - Insertamos 'when_iso' y 'modes' directamente en el texto de la query.
    - Las coordenadas van como variables.
    """
    
    # 1. Preparar string de modos: "{ mode: BUS }, { mode: SUBWAY }"
    modes_str = ", ".join([f"{{ mode: {m} }}" for m in modes])
    
    # 2. INYECCIÓN DE TEXTO (Aquí estaba el error antes)
    # Tenemos dos %s en la query. Pasamos una tupla con (fecha, modos)
    final_query = PLAN_QUERY % (when_iso, modes_str)

    # 3. Variables (Solo coordenadas, ya no va 'time')
    variables = {
        "oLat": float(o_lat), "oLon": float(o_lon),
        "dLat": float(d_lat), "dLon": float(d_lon)
    }

    # 4. Enviar
    data = otp_graphql(final_query, variables)
    
    if not data or "planConnection" not in data:
        return None

    edges = data["planConnection"]["edges"]
    if not edges:
        return None

    best_minutes = float('inf')
    found = False
    
    for e in edges:
        node = e["node"]
        start = dtparser.isoparse(node["start"])
        end = dtparser.isoparse(node["end"])
        minutes = (end - start).total_seconds() / 60.0
        
        if minutes < best_minutes:
            best_minutes = minutes
            found = True

    return best_minutes if found else None

print("✅ Motor OTP configurado: Hora y Modos insertados como texto fijo.")

✅ Motor OTP configurado: Hora y Modos insertados como texto fijo.


In [3]:
# ============================================================================
# CELDA 3: PREPARACIÓN DE PUNTOS (CENTROIDES COMUNALES)
# ============================================================================

# Cargar comunas
gdf_comunas = load_geodata(RUTA_GPKG, layer="comunas_rm_censo")

# Asegurar proyección Lat/Lon (WGS84) para OTP
comunas_wgs = gdf_comunas.to_crs(epsg=4326).copy()

# Calcular centroides
comunas_wgs["centroid"] = comunas_wgs.geometry.centroid

# Definir columnas de ID y Nombre (Basado en Notebook 01/02)
ID_COL = "CUT_COM" 
NAME_COL = "COMUNA"

# Crear tabla limpia de puntos
points = comunas_wgs[[ID_COL, NAME_COL, "centroid"]].copy()
points["lat"] = points["centroid"].y
points["lon"] = points["centroid"].x
points = points.drop(columns=["centroid"])

print(f"📍 Puntos de origen/destino preparados: {len(points)} comunas.")
display(points.head(3))

✅ Cargado Capa 'comunas_rm_censo': 52 registros | CRS: EPSG:32719
📍 Puntos de origen/destino preparados: 52 comunas.


,CUT_COM,COMUNA,lat,lon
0,13130,San Miguel,-33.499060,-70.651504
1,13118,Macul,-33.489309,-70.599913
2,13119,Maipú,-33.507027,-70.808888


In [4]:
# ============================================================================
# CELDA 4: CÁLCULO DE MATRIZ DE TIEMPOS (CON CACHE)
# ============================================================================

CACHE_PATH = OUTPUTS_DIR / "otp_od_matrix_comunas.csv"

# 1. Cargar Cache si existe
if CACHE_PATH.exists():
    od_df = pd.read_csv(CACHE_PATH)
    # Crear set de pares ya hechos para no repetir (Origen, Destino)
    done_pairs = set(zip(od_df["origin_id"], od_df["dest_id"]))
    print(f"✅ Cache encontrado: {len(od_df)} rutas ya calculadas.")
else:
    od_df = pd.DataFrame(columns=["origin_id", "origin_name", "dest_id", "dest_name", "minutes"])
    done_pairs = set()
    print("🆕 Iniciando cálculo desde cero.")

# 2. Convertir a lista de diccionarios para iterar rápido
rows = points.to_dict("records")

# Configuración de prueba (Pon None para correr todo)
MAX_TEST = None  # Cambia a 5 si quieres probar rápido antes de correr todo

subset_o = rows[:MAX_TEST] if MAX_TEST else rows
subset_d = rows[:MAX_TEST] if MAX_TEST else rows

new_records = []
save_interval = 20 # Guardar cada 20 consultas

print(f"🚀 Comenzando cálculo para {len(subset_o)} x {len(subset_d)} pares...")

start_time = time.time()

for i, origin in enumerate(subset_o):
    for dest in subset_d:
        
        # Identificador único del par
        pair_key = (origin[ID_COL], dest[ID_COL])
        
        # Si ya está hecho o es el mismo punto, saltar
        if pair_key in done_pairs:
            continue
        
        if origin[ID_COL] == dest[ID_COL]:
            minutes = 0.0 # Viaje a sí mismo
        else:
            # CONSULTA A OTP
            minutes = otp_travel_time_minutes(
                origin["lat"], origin["lon"], 
                dest["lat"], dest["lon"], 
                WHEN_ISO
            )
            
            # Feedback visual (print cada cierto tiempo)
            print(f"   Calculando: {origin[NAME_COL]} -> {dest[NAME_COL]} = {minutes if minutes else 'N/A'} min", end="\r")

        # Agregar a resultados
        new_records.append({
            "origin_id": origin[ID_COL],
            "origin_name": origin[NAME_COL],
            "dest_id": dest[ID_COL],
            "dest_name": dest[NAME_COL],
            "minutes": minutes
        })
        done_pairs.add(pair_key)

        # Guardado incremental
        if len(new_records) >= save_interval:
            chunk_df = pd.DataFrame(new_records)
            od_df = pd.concat([od_df, chunk_df], ignore_index=True)
            od_df.to_csv(CACHE_PATH, index=False)
            new_records = [] # Limpiar buffer

# Guardado final de remanentes
if new_records:
    od_df = pd.concat([od_df, pd.DataFrame(new_records)], ignore_index=True)
    od_df.to_csv(CACHE_PATH, index=False)

total_time = time.time() - start_time
print(f"\n✅ Proceso terminado en {total_time:.1f} segundos.")
print(f"💾 Archivo guardado: {CACHE_PATH}")
print(f"📊 Total pares procesados: {len(od_df)}")

KeyError: 'origin_id'

In [ ]:
# ============================================================================
# CELDA 5: CARGA DE SERVICIOS Y CREACIÓN DE ÍNDICE ESPACIAL (SINDEX)
# ============================================================================

# 1. Mapeo: Nombre amigable (para el código) -> Nombre real de la capa en el GeoPackage
SERVICE_LAYERS_MAP = {
    "salud": "establecimientos_salud",
    "educacion_escolar": "establecimientos_educacion",
    "educacion_superior": "establecimientos_educacion_superior",
    "carabineros": "cuarteles_carabineros",
    "bomberos": "companias_bomberos",
    "metro_tren": "paradas_metro_tren",
    "micro": "paradas_micro",
    "deporte_infra": "infraestructura_deportiva",
    "municipios": "municipios",
    "ferias_libres": "ferias_libres",
    "areas_verdes": "areas_verdes",
    "iglesias": "osm_iglesias",
    "museos": "osm_museos",
    "supermercados": "osm_supermercados",
    "almacenes_barrio": "osm_almacenes_barrio",
    "bancos": "osm_bancos",
    "malls": "osm_malls",
    "bencineras": "osm_bencineras",
    "estadios": "osm_estadios",
}

# 2. Generar lista de categorías objetivo automáticamente
# Toma todas las claves del diccionario anterior.
TARGET_CATEGORIES = list(SERVICE_LAYERS_MAP.keys())

print(f"✅ Configuración de servicios actualizada.")
print(f"🎯 Se calculará accesibilidad para {len(TARGET_CATEGORIES)} categorías.")
print(f"📋 Categorías: {TARGET_CATEGORIES}")

def load_services_unified(target_cats):
    gdfs = []
    print("📥 Cargando capas de servicios...")
    for cat in target_cats:
        layer = SERVICE_LAYERS_MAP.get(cat)
        if not layer: continue
        try:
            # Cargar y asegurar proyección Lat/Lon
            gdf = gpd.read_file(RUTA_GPKG, layer=layer).to_crs(4326)
            if gdf.empty: continue
            
            # Convertir polígonos a puntos para ruteo
            gdf["geometry"] = gdf.geometry.representative_point()
            gdf["categoria"] = cat
            gdfs.append(gdf[["categoria", "geometry"]])
        except Exception as e:
            print(f"⚠️ Error cargando {cat}: {e}")
            pass
            
    if not gdfs: return gpd.GeoDataFrame()
    return pd.concat(gdfs, ignore_index=True)

# 1. Cargar todo en un solo GeoDataFrame
servicios = load_services_unified(TARGET_CATEGORIES)

# 2. CREAR EL ÍNDICE ESPACIAL (La clave de la velocidad)
# Esto permite búsquedas instantáneas en vez de medir distancia uno por uno
if not servicios.empty:
    servicios_sindex = servicios.sindex
    print(f"✅ Servicios cargados: {len(servicios)} puntos.")
    print("✅ Índice espacial (R-tree) construido.")
else:
    print("❌ No se cargaron servicios. Revisa la ruta del GPKG.")

In [ ]:
# ============================================================================
# CELDA 6: BUSCAR EL MÁS CERCANO Y CALCULAR TIEMPO
# ============================================================================
from shapely.geometry import Point

def get_min_travel_time(origin_lat, origin_lon, category, k=5):
    # 1. Filtrar servicios
    subset = servicios_gdf[servicios_gdf["categoria"] == category].copy()
    if subset.empty: return None
    
    # 2. Distancia euclidiana rápida para pre-filtrar
    p_org = Point(origin_lon, origin_lat)
    subset["dist"] = subset.geometry.distance(p_org)
    candidates = subset.nsmallest(k, "dist")
    
    min_time = float('inf')
    found = False
    
    # 3. Consulta OTP real para los candidatos
    for _, row in candidates.iterrows():
        dest = row.geometry
        # Aquí llama a la función de la Celda 2 que arreglamos
        t = otp_travel_time_minutes(origin_lat, origin_lon, dest.y, dest.x, WHEN_ISO)
        
        if t is not None:
            found = True
            if t < min_time:
                min_time = t
                
    return min_time if found else None

print("✅ Lógica de accesibilidad lista.")

In [ ]:
# ============================================================================
# CELDA 7: EJECUCIÓN MASIVA
# ============================================================================
import time

ACC_PATH = OUTPUTS_DIR / "accesibilidad_servicios_otp.csv"

# Reanudar si existe
if ACC_PATH.exists():
    acc_df = pd.read_csv(ACC_PATH)
    done = set(zip(acc_df["cod_comuna"], acc_df["categoria"]))
    print(f"🔄 Reanudando: {len(acc_df)} datos previos.")
else:
    acc_df = pd.DataFrame(columns=["cod_comuna", "comuna", "categoria", "minutos"])
    done = set()
    print("🆕 Iniciando cálculo.")

# Usamos 'points' (definido en Celda 3)
origins = points.to_dict("records")
buffer = []

print(f"🚀 Procesando {len(origins)} comunas...")

for i, org in enumerate(origins):
    cid = org[ID_COL]
    cname = org[NAME_COL]
    
    print(f"[{i+1}/{len(origins)}] {cname}...", end="\r")
    
    for cat in TARGET_CATEGORIES:
        if (cid, cat) in done: continue
        
        try:
            val = get_min_travel_time(org["lat"], org["lon"], cat)
            buffer.append({"cod_comuna": cid, "comuna": cname, "categoria": cat, "minutos": val})
            done.add((cid, cat))
        except Exception as e:
            print(f"Err: {e}")

    # Guardar cada 5 comunas
    if len(buffer) >= 20:
        acc_df = pd.concat([acc_df, pd.DataFrame(buffer)], ignore_index=True)
        acc_df.to_csv(ACC_PATH, index=False)
        buffer = []

# Guardado final
if buffer:
    acc_df = pd.concat([acc_df, pd.DataFrame(buffer)], ignore_index=True)
    acc_df.to_csv(ACC_PATH, index=False)

print("\n✅ Proceso completado.")

In [ ]:
# ============================================================================
# CELDA 8: EXPORTAR PARA MACHINE LEARNING
# ============================================================================

df = pd.read_csv(ACC_PATH)

# Pivotar para tener una columna por servicio
df_pivot = df.pivot_table(index=["cod_comuna", "comuna"], columns="categoria", values="minutos").reset_index()

# Prefijo 'acc_' para diferenciar de los conteos
cols_map = {c: f"acc_{c}" for c in TARGET_CATEGORIES}
df_pivot = df_pivot.rename(columns=cols_map)

# Guardar Parquet (Más rápido para Pandas) y CSV
df_pivot.to_parquet(OUTPUTS_DIR / "comunas_accessibility_otp.parquet")
df_pivot.to_csv(OUTPUTS_DIR / "comunas_accessibility_otp.csv", index=False)

print("📦 Datos listos para Notebook 04:")
display(df_pivot.head())

In [ ]:
# ============================================================================
# CELDA 9: FUNCIONES AVANZADAS (CACHE + SINDEX) - CORREGIDA
# ============================================================================
import hashlib
import json
import geopandas as gpd
from shapely.geometry import Point

# Usa UTM 19S para medir metros reales en Santiago
METRIC_CRS = "EPSG:32719"

# Carpeta para guardar respuestas individuales de OTP
OTP_CACHE_DIR = OUTPUTS_DIR / "otp_cache_json"
OTP_CACHE_DIR.mkdir(parents=True, exist_ok=True)

def _otp_cache_key(o_lat, o_lon, d_lat, d_lon, when_iso):
    s = f"{o_lat:.6f},{o_lon:.6f}->{d_lat:.6f},{d_lon:.6f}|{when_iso}"
    return hashlib.md5(s.encode("utf-8")).hexdigest()

def otp_travel_time_minutes_cached(o_lat, o_lon, d_lat, d_lon, when_iso):
    """
    Consulta OTP con sistema de archivos caché.
    """
    key = _otp_cache_key(o_lat, o_lon, d_lat, d_lon, when_iso)
    path = OTP_CACHE_DIR / f"{key}.json"

    # 1. Leer del disco
    if path.exists():
        try:
            with open(path, "r", encoding="utf-8") as f:
                data = json.load(f)
            return data["minutes"]
        except:
            pass 

    # 2. Consultar OTP (función de Celda 2)
    minutes = otp_travel_time_minutes(o_lat, o_lon, d_lat, d_lon, when_iso)

    # 3. Guardar en disco
    if minutes is not None:
        with open(path, "w", encoding="utf-8") as f:
            json.dump({"minutes": minutes}, f)

    return minutes

def query_point_access_otp_smart(lat, lon, categories, radius_m=5000, k=5):
    """
    Usa el índice espacial para encontrar candidatos y luego OTP con cache.
    """
    origin_wgs = Point(lon, lat)
    
    # --- FIX AQUÍ ---
    # En lugar de extraer el objeto con .iloc[0], mantenemos la GeoSeries
    # para poder usar .to_crs() y .buffer() en cadena.
    
    # 1. Crear GeoSeries del origen
    gs_origin = gpd.GeoSeries([origin_wgs], crs=4326)
    
    # 2. Transformar a métrico -> Buffer -> Volver a WGS84
    # Esto devuelve una GeoSeries con 1 polígono (el círculo transformado)
    gs_buffer_wgs = gs_origin.to_crs(METRIC_CRS).buffer(radius_m).to_crs(4326)
    
    # 3. Obtener el Bounding Box (minx, miny, maxx, maxy) del polígono
    # .total_bounds devuelve el array directamente
    bbox = gs_buffer_wgs.total_bounds 
    # ----------------
    
    # 1. FILTRO RÁPIDO con Sindex (Intersección con la caja del buffer)
    possible_inds = list(servicios_sindex.intersection(bbox))
    candidates = servicios.iloc[possible_inds].copy()
    
    results = []
    
    for cat in categories:
        # Filtrar por categoría
        subset = candidates[candidates["categoria"] == cat].copy()
        if subset.empty:
            results.append({"categoria": cat, "tiempo_min": None})
            continue
            
        # Filtrar los k más cercanos por distancia (pre-filtro)
        subset["dist"] = subset.geometry.distance(origin_wgs)
        top_k = subset.nsmallest(k, "dist")
        
        min_minutes = float('inf')
        found = False
        
        # 2. CONSULTA OTP REAL (con Cache)
        for _, service in top_k.iterrows():
            dest = service.geometry
            t = otp_travel_time_minutes_cached(lat, lon, dest.y, dest.x, WHEN_ISO)
            
            if t is not None:
                found = True
                if t < min_minutes:
                    min_minutes = t
        
        results.append({
            "categoria": cat, 
            "tiempo_min": min_minutes if found else None
        })
        
    return pd.DataFrame(results)

print("✅ Funciones Smart corregidas (Fix to_crs).")

In [ ]:
# ============================================================================
# CELDA 10: CÁLCULO DE ACCESIBILIDAD (VERSIÓN FINAL CONECTADA A DOCKER)
# ============================================================================
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import requests
import time

# --- CONFIGURACIÓN CLAVE ---
# 1. URL: Dentro de Docker, OTP se llama "otp", no "localhost"
OTP_URL = "http://otp:8080/otp/routers/default/plan"

# 2. FECHA: ¡Ajusta esto a una fecha válida de tu GTFS! (Ej: un martes hábil)
FECHA_VIAJE = "2023-11-15"   
HORA_VIAJE = "08:00am"

# 3. PARAMETROS DE BÚSQUEDA
RADIUS_M = 2000      # Radio de búsqueda de servicios (metros)
K_NEIGHBORS = 1      # Solo el más cercano para hacerlo rápido
# ---------------------------

print(f"🔧 Preparando geometrías y conectando a {OTP_URL}...")

# 1. PREPARACIÓN DE ENTORNOS (Proyecciones)
# Transformamos a Metros (EPSG:3857) para medir distancias exactas
comunas_m = gpd.GeoDataFrame(
    points, 
    geometry=gpd.points_from_xy(points["lon"], points["lat"]), 
    crs=4326
).to_crs(3857)

servicios_m = servicios.to_crs(3857)
servicios_sindex_m = servicios_m.sindex

# Aseguramos IDs únicos
servicios = servicios.reset_index(drop=True)
servicios_m["id_interno"] = servicios_m.index

# 2. FUNCIÓN PARA CONSULTAR A OTP
def get_otp_time(orig_lat, orig_lon, dest_lat, dest_lon, mode_str):
    params = {
        "fromPlace": f"{orig_lat},{orig_lon}",
        "toPlace": f"{dest_lat},{dest_lon}",
        "time": HORA_VIAJE,
        "date": FECHA_VIAJE,
        "mode": mode_str,
        "maxWalkDistance": "5000",
        "arriveBy": "false"
    }
    try:
        # Timeout corto (2s) porque están en la misma red local
        response = requests.get(OTP_URL, params=params, timeout=2)
        
        if response.status_code == 200:
            data = response.json()
            # Navegamos el JSON para buscar la duración
            if 'plan' in data and 'itineraries' in data['plan'] and len(data['plan']['itineraries']) > 0:
                duration_sec = data['plan']['itineraries'][0]['duration']
                return duration_sec / 60.0 # Retornar en minutos
            elif 'error' in data:
                # Si OTP responde pero dice "No trip found"
                return -1 
        return None # Error de conexión o servidor
    except Exception:
        return None

# 3. BUCLE PRINCIPAL
resultados = []
print(f"🚀 Iniciando cálculo para {len(comunas_m)} orígenes...")

for idx, row_orig in comunas_m.iterrows():
    # Datos del origen
    cod_origen = points.iloc[idx][ID_COL] # Asegúrate que ID_COL esté definido antes
    orig_lat = points.iloc[idx]["lat"]
    orig_lon = points.iloc[idx]["lon"]
    
    # Imprimir progreso cada 10 filas para no saturar
    if idx % 10 == 0:
        print(f"   ... Procesando fila {idx}/{len(comunas_m)}")

    # FILTRO ESPACIAL: Primero buscamos lo que está cerca geométricamente
    centroide_m = row_orig.geometry
    bbox = centroide_m.buffer(RADIUS_M).bounds
    posibles_idxs = list(servicios_sindex_m.intersection(bbox))
    
    # Si no hay nada cerca, saltamos
    if not posibles_idxs:
        for cat in TARGET_CATEGORIES:
            resultados.append({"cod": cod_origen, "cat": cat, "minutos": None, "estado": "Fuera de rango"})
        continue

    candidatos_m = servicios_m.iloc[posibles_idxs].copy()
    
    # Iteramos por cada categoría de servicio (ej: Salud, Educación, Paraderos)
    for categoria in TARGET_CATEGORIES:
        subset = candidatos_m[candidatos_m["categoria"] == categoria]
        
        if subset.empty:
            resultados.append({"cod": cod_origen, "cat": categoria, "minutos": None, "estado": "Sin servicio cercano"})
            continue

        # DEFINIR MODO DE TRANSPORTE
        # Si buscamos paraderos, vamos caminando. Si buscamos hospitales, tomamos micro.
        if categoria in ["micro", "metro_tren", "paradas_micro"]:
            otp_mode = "WALK"
        else:
            otp_mode = "WALK,TRANSIT"

        # Tomamos los K más cercanos geométricamente para preguntar a OTP
        # (Esto ahorra consultas innecesarias a servicios lejanos)
        subset["dist_geo"] = subset.geometry.distance(centroide_m)
        top_k = subset.sort_values("dist_geo").head(K_NEIGHBORS)
        
        tiempos_otp = []
        
        for _, svc_row in top_k.iterrows():
            # Volvemos a Lat/Lon para la API (usando el GeoDataFrame original 'servicios' que está en 4326)
            dest_lat = servicios.iloc[svc_row.name].geometry.y
            dest_lon = servicios.iloc[svc_row.name].geometry.x
            
            t = get_otp_time(orig_lat, orig_lon, dest_lat, dest_lon, otp_mode)
            
            if t is not None and t >= 0:
                tiempos_otp.append(t)
        
        # Guardamos el mejor tiempo encontrado
        if tiempos_otp:
            resultados.append({
                "cod": cod_origen, 
                "cat": categoria, 
                "minutos": min(tiempos_otp),
                "estado": "OK"
            })
        else:
            resultados.append({
                "cod": cod_origen, 
                "cat": categoria, 
                "minutos": None, 
                "estado": "Error OTP o Sin Ruta"
            })

# 4. EXPORTAR RESULTADOS
df_res = pd.DataFrame(resultados)
print("\n✅ Cálculo terminado.")
print(df_res.head())

# Guardar
# Asegúrate que OUTPUTS_DIR esté definido, si no, usa ruta relativa
try:
    df_res.to_csv(OUTPUTS_DIR / "accesibilidad_otp_final.csv", index=False)
    print(f"📁 Guardado en {OUTPUTS_DIR}")
except:
    df_res.to_csv("accesibilidad_otp_final.csv", index=False)
    print("📁 Guardado en carpeta actual.")

In [ ]:
# ============================================================================
# CELDA 11: PIVOT, LIMPIEZA Y RECUPERACIÓN DE COMUNAS (FIX 52/52)
# ============================================================================
# Definir la variable que falta
OUT_LONG = OUTPUTS_DIR / "acc_long_partial.csv"
# 1. Cargar resultados parciales
if not OUT_LONG.exists():
    raise FileNotFoundError("❌ No se encontró el archivo 'acc_long_partial.csv'.")

df_long = pd.read_csv(OUT_LONG)

# 2. Pivotar (Largo -> Ancho)
acc_wide_raw = df_long.pivot_table(
    index=["cod_comuna"], # Usamos solo el código como índice primario
    columns="categoria",
    values="tiempo_min",
    aggfunc="min"
).reset_index()

# 3. Renombrar columnas de categorías
acc_wide_raw.columns.name = None
cat_cols = [c for c in acc_wide_raw.columns if c != "cod_comuna"]
rename_map = {c: f"acc_{c}" for c in cat_cols}
acc_wide_raw = acc_wide_raw.rename(columns=rename_map)

# ----------------------------------------------------------------------------
# 4. CRUCIAL: RECUPERAR LAS COMUNAS PERDIDAS (LEFT JOIN)
# ----------------------------------------------------------------------------
# Tomamos la tabla maestra de puntos (52 comunas) como base
base_comunas = points[[ID_COL, NAME_COL]].copy()
base_comunas = base_comunas.rename(columns={ID_COL: "cod_comuna", NAME_COL: "comuna"})

# Pegamos los resultados. Las que no tengan datos quedarán como NaN automágicamente.
acc_final = base_comunas.merge(acc_wide_raw, on="cod_comuna", how="left")

# 5. Guardar
OUT_FINAL_PARQUET = OUTPUTS_DIR / "comunas_accessibility_otp.parquet"
OUT_FINAL_CSV = OUTPUTS_DIR / "comunas_accessibility_otp.csv"

acc_final.to_parquet(OUT_FINAL_PARQUET, index=False)
acc_final.to_csv(OUT_FINAL_CSV, index=False)

print("\n📊 RESULTADO FINAL CORREGIDO:")
display(acc_final.head())

print("-" * 30)
print(f"✅ Archivos guardados en: {OUTPUTS_DIR}")
print(f"🧩 Dimensiones esperadas: (52, X) -> Dimensiones reales: {acc_final.shape}")

# Verificar si hay Nulos (Las comunas que recuperamos tendrán nulos)
nulos = acc_final.isnull().sum()
print("\n🔍 Conteo de comunas sin cobertura (NaN) por servicio:")
print(nulos[nulos > 0])